# Supervised Classsification

There are many different types of models we can use for classification,
each of these models have types of problems they are well suited to.
The goal of this notebook is to identify algorithms 
that will effectively classify our dataset
which we can then investigate further.

In [12]:
import numpy as np
from sklearn import cluster, decomposition, preprocessing
from sdanalysis.order import compute_neighbours, relative_orientations, relative_distances

import warnings
warnings.filterwarnings('ignore')

# Import project tools
import sys
sys.path.append('../src')
from trimer import plot_clustering, read_files, classify_mols

# Configure Bokeh to output the figures to the notebook
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

In [13]:
crystals=["p2", "p2gg", "pg"]
snaps = read_files(
    temperature=0.40,
    pressure=1.0,
    crystals=crystals,
)

### Loading the training data

We need to load in the training dataset we created in the first notebook.
At this point we are interested in two sets of data,

- $X$, the input data which is the orientation of the six nearest neighbours
- $y$, the true labelled classification of the data.

In [14]:
classes = np.concatenate(
    [classify_mols(snap, crys) for snap, crys in zip(snaps, crystals)]
)

In [15]:
orientations = np.concatenate(
    [relative_orientations(s.box, s.position, s.orientation) for s in snaps]
)

In [16]:
mask = classes < 4
X = orientations[mask]
y = classes[mask]

### Collating the models

The first step here is creating a list of models we would like to test.
An excellent property of scikit-learn is 
that all the algorithms have the same API,
allowing us to treat them all in the same way. 

This is not an exhastive list of all the possible classifiers in scikit-learn,
just a smattering for comparison.
For a more exhastive list check out [the scikit-learn documentation](http://scikit-learn.org/stable/supervised_learning.html#supervised-learning),
and feel free to add more to the list.

In [21]:
import sklearn

ml_models = {
    'LR': sklearn.linear_model.LogisticRegression(),
    'SGD': sklearn.linear_model.SGDClassifier(tol=1e-3, max_iter=1000),
    'LDA': sklearn.discriminant_analysis.LinearDiscriminantAnalysis(),
    'DT': sklearn.tree.DecisionTreeClassifier(),
    'KNN': sklearn.neighbors.KNeighborsClassifier(),
    'NB': sklearn.naive_bayes.GaussianNB(),
    'SVM': sklearn.svm.SVC(),
    'NN': sklearn.neural_network.MLPClassifier()
}

### Testing the Models

With a collection of models to test,
we now need some method of testing the models to compare them.
To perform the initial screening of datasets
we are going to break our training data into two groups,

- the training set, comprising 80% of the molecules
- the validation set, comprising the remaining 20%.

This division of the dataset gives us a set of data 
previously unseen by the algorithms,
giving us a method of testing whether
the algorithm is acutally learning the underlying features,
or just 'memorising' the training data.
This division of data will be through a random selection
so as not to bias the selection of data.

In [22]:
from sklearn import model_selection

validation_size = 0.20
seed = 7

selected = model_selection.train_test_split(X, y, test_size=validation_size, random_state=seed)
X_train, X_validation, y_train, y_validation = selected

To get an idea of the models which
warrant a further investigation,
we can iterate through each of our models.
Each model is scored by breaking the training data into `n_splits`,
using one of these splits for testing and
the remaining splits for training.
This process is referred to as *cross validation*
and typically the number of splits is 10.
For the purposes of this running in a reasonable amount of time,
`n_splits` is set to 2.

In [24]:
scoring='accuracy'
n_splits = 2
# Typically n_splits would be 10 but it runs much slower
#n_splits = 10

# Iterate through each model in our dictionary of models
for name, model in ml_models.items():
    kfold = model_selection.KFold(n_splits=n_splits, random_state=seed)
    cv_results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    print(f'{name:5s}: {cv_results.mean():.5f} ± {cv_results.std():.5f}')

LR   : 0.82357 ± 0.00118
SGD  : 0.76783 ± 0.00165
LDA  : 0.77633 ± 0.00307
DT   : 0.88262 ± 0.00118
KNN  : 0.90482 ± 0.00165
NB   : 0.82664 ± 0.00803
SVM  : 0.85002 ± 0.00118
NN   : 0.90812 ± 0.00213


Out of all the algorithms tested, 
there are three that stand out 

- K-Nearest Neighbours (KNN),
- Support Vector Machine (SVM), and
- Neural Network (NN)

with accuracies in excess of 95%. 

So with these three algorithms it is likely worth
tweaking the algorithms slightly from 
the defualt paramters in an effort to improve performance.
It is also worth understanding which classes
each of these algorithms is strongest at classifying.
For this additional data we are going to be using a [confusion matrix](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html).
In a confusion matrix, 
the diagonal elements represent the correct classifications,
while the off diagonal elements are the values 
which were incorrectly classified.

It is interesting to note that all of the models
have the most difficulty with the liquid/crystal characterisation,
with the largest proportion of false positives being 
crystal incorrectly classified as liquid. 
To make this model we have created persistent
it needs to be saved which is done using `joblib`.

In [31]:
from sklearn.externals import joblib
knn = ml_models['KNN']
knn.fit(X_train, y_train)
joblib.dump(knn, '../models/knn-trimer.pkl') 

['../models/knn-trimer.pkl']